In [1]:
%matplotlib inline

# 保存并加载模型

在本单元中，我们将了解如何通过保存、加载和运行模型预测来保持模型状态。


In [2]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## 保存和加载模型权重

PyTorch 模型将学习到的参数存储在一个名为“state_dict”的内部状态字典中。 这些可以通过 `torch.save` 方法持久化：



In [3]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'data/model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/lokinfey/.cache/torch/checkpoints/vgg16-397923af.pth
100.0%


To load model weights, you need to create an instance of the same model first, and then load the parameters using the `load_state_dict()` method.



In [4]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('data/model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

> **Note:** Be sure to call `model.eval()` method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.


## 保存和加载带有形状的模型

在加载模型权重时，我们需要先实例化模型类，因为该类定义了网络的结构。 我们可能希望将此类的结构与模型一起保存，在这种情况下，我们可以将 `model`（而不是 `model.state_dict()`）传递给保存函数：

In [5]:
torch.save(model, 'data/vgg_model.pth')

We can then load the model like this:

In [6]:
model = torch.load('data/vgg_model.pth')

> **Note:** This approach uses Python [pickle](https://docs.python.org/3/library/pickle.html) module when serializing the model, thus it relies on the actual class definition to be available when loading the model.


## 将模型导出到 ONNX

PyTorch 还具有本机 ONNX 导出支持。 然而，鉴于 PyTorch 执行图的动态特性，导出过程必须遍历执行图以生成持久化的 ONNX 模型。 出于这个原因，应该将适当大小的测试变量传递给导出例程（在我们的例子中，我们将创建一个正确大小的虚拟零张量）：

In [7]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'data/model.onnx')

您可以使用 ONNX 模型做很多事情，包括在不同平台和不同编程语言上运行推理。